##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 初心者のための TensorFlow 2.0 入門

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

このガイドでは簡単な導入として、[Keras](https://www.tensorflow.org/guide/keras/overview) を使って次のことを行います。

1. 画像を分類するニューラルネットワークを構築する。
2. そのニューラルネットワークをトレーニングする。
3. 最後に、モデルの精度を評価する。
4. モデルの精度を評価します。

このファイルは [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) の notebook ファイルです。 Python プログラムはブラウザ上で直接実行されます。TensorFlow を学んだり使ったりするには最良の方法です。Google Colab のnotebook の実行方法は以下のとおりです。

1. Pythonランタイムへの接続：メニューバーの右上で「接続」を選択します。
2. ノートブックのすべてのコードを実行するには、**Runtime** &gt; **Run all** を選択します。コードセルを 1 つずつ実行するには、セルにマウスポインタ―を合わせて **Run cell** アイコンを選択します。

![Run cell アイコン](https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/quickstart/images/beginner/run_cell_icon.png?raw=1)

## TensorFlow をセットアップする

まず、TensorFlow をプログラムにインポートします。

In [2]:
import tensorflow as tf

[Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb) ではなく独自の開発環境で手順を実行している場合は、開発用 TensorFlow のセットアップについて、[インストールガイド](https://www.tensorflow.org/install)をご覧ください。

注意: 独自の開発環境を使用している場合は、TensorFlow 2 パッケージをインストールする際に、最新の `pip` にアップグレード済みであることを確認してください。詳細は、[インストールガイド](https://www.tensorflow.org/install)をご覧ください。

## データセットを読み込む

[MNIST データセット](http://yann.lecun.com/exdb/mnist/)を読み込んで準備します。画像のピクセル値は 0～255 です。値を `255.0` で割って、0～1 の範囲に変換しましょう。また、この操作によって、サンプルデータが整数から浮動小数点にも変換されます。

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## 機械学習モデルを構築する

`tf.keras.Sequential` モデルを構築します。

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


レイヤーごとに 1 つの入力[テンソル](https://www.tensorflow.org/guide/tensor)と 1 つの出力テンソルを持つ複数のレイヤーをスタックするには、[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) が便利です。レイヤーは、既知の数学的構造を持つ関数であり、再利用することが可能で、トレーニング可能な変数を含むことができます。ほとんどの TensorFlow モデルはレイヤーで構成されています。このモデルでは、[`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten)（平坦化された）[`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)（密な）[`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout)（ドロップアウト）レイヤーが使用されています。

各サンプルについて、モデルは "[logits](https://developers.google.com/machine-learning/glossary#logits)" または "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" スコアのベクトルをクラスごとに返します。

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.09902991, -0.10177586, -0.10898976, -0.5258892 , -0.31688353,
        -0.34099534, -0.5407501 , -0.04493963, -0.2759031 , -0.29194123]],
      dtype=float32)

`tf.nn.softmax` 関数は、クラスごとにこれらのロジットを "probabilities" に変換します。

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.116412  , 0.11609277, 0.11525831, 0.07596527, 0.09362355,
        0.09139312, 0.0748447 , 0.12288217, 0.09753999, 0.0959881 ]],
      dtype=float32)

注意: この `tf.nn.softmax` をネットワークの最後のレイヤーのアクティベーション関数として組み込むことも可能です。こうすることでモデルの出力をより直接的に解釈可能にすることもできますが、softmax 出力を使用する場合、すべてのモデルに対して正確で数値的に安定した損失計算を提供することは不可能であるため、この方法は推奨されません。

`losses.SparseCategoricalCrossentropy` を使用して、トレーニングの損失関数を定義します。

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

損失関数は、グラウンドトゥルース値のベクトルとロジットのベクトルを取り、サンプルごとにスカラー損失を返します。この損失は true クラスの負の対数確率と同等です。モデルが確実に正しいクラスである場合、損失はゼロとなります。

トレーニングされていないこのモデルでは、ランダムに近い確率（クラス当たり 1/10）が得られるため、最初の損失は `-tf.math.log(1/10) ~= 2.3` に近くなります。

In [8]:
loss_fn(y_train[:1], predictions).numpy()

np.float32(2.3925853)

トレーニングを開始する前に、Keras `Model.compile` を使用してモデルの構成とコンパイルを行います。[`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) クラスを `adam` に、`loss` を前に定義した `loss_fn` 関数に設定し、`metrics` パラメータを `accuracy` に設定して評価するモデルの指標を指定します。

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## モデルをトレーニングして評価する

損失を最小限に抑えられるようにモデルのパラメータを `Model.fit` メソッドで調整します。

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8608 - loss: 0.4777
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9552 - loss: 0.1518
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9672 - loss: 0.1103
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9743 - loss: 0.0863
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9778 - loss: 0.0743


`Model.evaluate` メソッドは、通常[検証セット](https://developers.google.com/machine-learning/glossary#validation-set)または[テストセット](https://developers.google.com/machine-learning/glossary#test-set)でモデルのパフォーマンスをチェックします。

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - 4ms/step - accuracy: 0.9780 - loss: 0.0696


[0.0695667639374733, 0.9779999852180481]

`tf.nn.softmax` 関数はクラスごとにこれらのロジットを "確率" に変換します。

モデルが確率を返すようにするには、トレーニング済みのモデルをラップして、それに softmax を接続することができます。

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[5.0501242e-08, 7.2381336e-09, 6.0068564e-06, 5.3816853e-05,
        2.2509968e-10, 6.6089029e-08, 1.2178268e-13, 9.9993682e-01,
        2.7379247e-07, 2.9286671e-06],
       [4.6408400e-07, 1.8403535e-06, 9.9999261e-01, 6.9540152e-07,
        2.7457018e-15, 2.3920856e-08, 7.1331959e-07, 5.0215336e-15,
        3.7193931e-06, 3.9951249e-15],
       [5.5575015e-07, 9.9915290e-01, 6.9965565e-05, 5.7896659e-07,
        1.0220055e-04, 1.0654214e-05, 2.1192820e-05, 5.6275155e-04,
        7.8235033e-05, 9.5091372e-07],
       [9.9927813e-01, 3.0345382e-10, 7.8117642e-05, 1.1796322e-06,
        1.6903972e-05, 6.9393609e-06, 4.3947148e-04, 1.7601060e-04,
        5.6287139e-08, 3.2192982e-06],
       [6.1793526e-06, 1.5202902e-08, 7.0382333e-07, 2.1902450e-08,
        9.9620962e-01, 4.4673817e-07, 9.6150106e-06, 3.9622864e-06,
        2.8740933e-06, 3.7666052e-03]], dtype=float32)>

## まとめ

おめでとうございます！事前構築済みのデータセットと [Keras](https://www.tensorflow.org/guide/keras/overview) API を使用して、機械学習モデルをトレーニングしました。

Keras のその他の使用例については、[チュートリアル](https://www.tensorflow.org/tutorials/keras/)をご覧ください。Keras を使ったモデルの構築に関してさらに学習するには、[ガイド](https://www.tensorflow.org/guide/keras)をお読みください。データの読み込みと準備についてさらに学習するには、[画像データの読み込み](https://www.tensorflow.org/tutorials/load_data/images)または [CSV データの読み込み](https://www.tensorflow.org/tutorials/load_data/csv)に関するチュートリアルをご覧ください。
